### Notebook options

In [1]:
%load_ext autoreload
%autoreload 2

### Imports

In [2]:
# importing standard libraries
import os
import json
import pandas as pd
import uuid

In [3]:
import duckdb

In [12]:
# Hyper Parameters

In [13]:
SCORING_CHUNK_MOD = 100
MIN_SUPPORT = 0.001
MIN_FEATURES_IN_RULE = 3
MAX_FEATURES_IN_RULE = 3
QUERIES_PER_CHUNK = 100
MIN_CONFIDENCE = 0.1
MIN_OBS = 5
MIN_LIFT = 1.1
MAX_RULES_PER_PRODUCT = 5000
QUANTILES_FOR_PLOTS = 10
MAX_OPTIONS_PER_CUSTOMER = 3

In [14]:
TARGET = "BENEFIT_OPTION"
TARGET_PREFIX = "T:"
FEATURES = ["*"]

In [15]:
import ibis

In [16]:
con = ibis.duckdb.connect("test_db_2.ddb")

In [17]:
customer_data = pd.read_csv('../data/01_raw/customer_data.csv')
product_data = pd.read_csv('../data/01_raw/product_data.csv')

In [18]:
customer_data.head()

,Customer ID,Grade,Sub Grade,Employment Length,Home Ownership,Annual Income,Loan Status,Payment Plan
0,1,A,A1,1.0,MORTGAGE,34000.0,Fully Paid,n
1,2,A,A1,1.0,MORTGAGE,36000.0,Fully Paid,n
2,3,A,A1,1.0,MORTGAGE,42000.0,Charged Off,n
3,4,A,A1,1.0,MORTGAGE,49000.0,Fully Paid,n
4,5,A,A1,1.0,MORTGAGE,50000.0,Fully Paid,n


In [19]:
from sklearn.pipeline import Pipeline

In [20]:
# Import the appropriate transformer from sklearn to group infrequent values
# in the categorical columns
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Now, construct the pipeline that takes the categorical columns and
# replaces infrequent values with the string 'Other'

# First, define the function that will be used to replace infrequent values
# with 'Other'

def replace_infrequent(x):
    counts = x.value_counts()
    return x.where(x.isin(counts[counts >= MIN_OBS].index), 'Other')

# Now, define the pipeline

infrequent_cat_pipeline = Pipeline([
    ('infrequent_cat_replacer', FunctionTransformer(replace_infrequent))
])

In [22]:
product_data.head()

,Customer ID,Product,Take Up,Value
0,6,Car,False,5926.968276
1,11,Car,False,12939.951513
2,17,Car,False,39558.648292
3,34,Car,False,86960.194717
4,40,Car,True,39107.895244


In [29]:
customer_data = con.read_csv('../data/01_raw/customer_data.csv', table_name='customer_data').execute()
product_data = con.read_csv('../data/01_raw/product_data.csv', table_name='product_data').execute()

In [ ]:
class IbisPipe():
    def __init__(self)

In [33]:
type(con)

ibis.backends.duckdb.Backend

In [129]:
type(t)

ibis.expr.types.relations.Table

In [159]:
class IbisCategoricalTransformer:
    def __init__(
            self,
            con: ibis.backends.duckdb.Backend,
            table: ibis.expr.types.relations.Table,
            column_name: str,
            infrequent_percentage: float = 0.01
    ):
        self.con = con
        self.table = table
        self.column_name = column_name
        self.infrequent_percentage = infrequent_percentage

    def fit(self):
        t = self.table
        # Get the frequency of each value
        freq = t.group_by(self.column_name).count().execute()
        freq.columns = ['value', 'count']
        freq['percentage'] = freq['count'] / freq['count'].sum()
        # Get the infrequent values
        self.frequent_values = freq[freq['percentage'] >= self.infrequent_percentage]['value'].tolist()
        return self
    
    def build_case_statement_element(self, case_statement, value, reference):
        return case_statement.when(value, reference)

    def build_case_statement(self, reference):
        case_statement = reference.case()
        for frequent_value in self.frequent_values:
            case_statement = self.build_case_statement_element(
                case_statement, frequent_value, reference
            )
        case_statement = case_statement.else_("_other").end()
        return case_statement

    def transform(self, table: ibis.expr.types.relations.Table):
        t = table
        # Replace the infrequent values with the string 'infrequent'
        t = t.mutate(**{
            self.column_name: self.build_case_statement(t[self.column_name])
        })
        return t

In [229]:
class IbisBinningTransformer:
    """
    A class for binning a column in an Ibis table based on quantiles.

    Args:
        con (ibis.backends.duckdb.Backend): The Ibis backend connection.
        table (ibis.expr.types.relations.Table): The Ibis table containing the data.
        column_name (str): The name of the column to be binned.
        n_bins (int, optional): The number of bins to create. Default is 10.

    Attributes:
        con (ibis.backends.duckdb.Backend): The Ibis backend connection.
        table (ibis.expr.types.relations.Table): The Ibis table containing the data.
        column_name (str): The name of the column to be binned.
        n_bins (int): The number of bins to create.
        bin_edges (list): The quantile-based bin edges.
    """

    def __init__(
        self,
        con: ibis.backends.duckdb.Backend,
        table: ibis.expr.types.relations.Table,
        column_name: str,
        n_bins: int = 10,
    ):
        self.con = con
        self.table = table
        self.column_name = column_name
        self.n_bins = n_bins
        self.bin_edges = None


    def bin_label_element(self, statement, bin_label):
        return statement.when(bin_label, bin_label)

    def fit(self):
        """
        Fit the binning transformer to the specified column based on quantiles.

        This method calculates bin edges based on quantiles of the data in the specified column.

        Returns:
            None
        """
        # Get the specified column from the table
        column = self.table[self.column_name]

        # Calculate quantile-based bin edges
        quantiles = [i / self.n_bins for i in range(1, self.n_bins)]
        self.bin_edges = column.quantile(quantiles).execute()
        return self

    def transform(self, table: ibis.expr.types.relations.Table = None):
        """
        Apply the quantile-based binning transformation to the specified column.

        This method assigns bin labels to the data in the specified column based on
        the quantile-based bin edges calculated during the fitting process.

        Returns:
            ibis.expr.api.NumericValue: A new Ibis expression representing the binned column.
        """
        if table is None:
            t = self.table
        else:
            t = table
        if self.bin_edges is None:
            raise ValueError("Fit method must be called before transform.")

        # Get the specified column from the table
        column = t[self.column_name]

        # Use ibis case statement to assign bin labels based on quantiles
        bin_labels = []
        for i, bin_edge in enumerate(self.bin_edges):
            bin_label = f"Bin_{i}"
            if i == 0:
                # Handle the first bin separately
                bin_labels.append(
                    ibis.case()
                    .when(column <= bin_edge, bin_label)
                    .else_(None)
                    .end()
                )
            elif i == len(self.bin_edges) - 1:
                # Handle the last bin separately
                bin_labels.append(
                    ibis.case()
                    .when(column > self.bin_edges[i - 1], bin_label)
                    .else_(None)
                    .end()
                )
            else:
                bin_labels.append(
                    ibis.case()
                    .when((column > self.bin_edges[i - 1]) & (column <= bin_edge), bin_label)
                    .else_(None)
                    .end()
                )

        # Concatenate bin labels into a single expression
        binned_column = None

        for bin_label in bin_labels:
            if binned_column is None:
                binned_column = bin_label
            else:
                binned_column = binned_column.fillna(bin_label)

        t = self.table
        # Replace the infrequent values with the string 'infrequent'
        t = t.mutate(**{
            self.column_name: binned_column
        })

        return t

In [230]:
class IbisPipe:
    def __init__(self, con: ibis.backends.duckdb.Backend):
        self.con = con
        self.steps = []
    
    def add_step(self, step):
        self.steps.append(step)
        return self
    
    def fit(self):
        for step in self.steps:
            step.fit()
        return self
    
    def transform(self):
        t = None
        for step in self.steps:
            if t is None:
                t = step.transform(step.table)
            else:
                t = step.transform(t)
        return t

In [231]:
pipe = IbisPipe(con)
CATEGORICAL_COLUMNS = ['Grade', 'Sub Grade', 'Home Ownership', 'Loan Status', 'Payment Plan']
INFREQUENT_PERCENTAGE = 0.01

In [232]:
t = con.table('customer_data')
for c in CATEGORICAL_COLUMNS:
    pipe = pipe.add_step(IbisCategoricalTransformer(con, t, c, INFREQUENT_PERCENTAGE))

In [233]:
bin = IbisBinningTransformer(con, t, 'Annual Income', 10)

In [234]:
bin = bin.fit()

In [235]:
bin.transform()

AttributeError: module 'ibis' has no attribute 'concat'. 

In [169]:
pipe = pipe.fit()

In [170]:
t = pipe.transform()

In [28]:
con.list_tables()

['customer_data']

In [6]:



#os.environ['UI_ENV'] = UI_ENV_NOTEBOOK #TODO! no value # TODO set in devcontainer.json

# importing local libraries
os.chdir("../src/")
import smartleads
from smartleads.util.setup_bigquery import create_datasets
from smartleads.util.tools import *
import smartleads.util.bq as bq
import smartleads.profiling
import smartleads.preprocessing
import smartleads.support
from smartleads.globals import *
import smartleads.id_agg
import smartleads.combo_generation
import smartleads.lift
import smartleads.score
import smartleads.combo_count
from smartleads.insights import (
    get_sample_associations_df, 
    get_target_dataframe, 
    get_included_dataframe,
    get_lift_distribution_dataframe,
    get_confidence_distribution_dataframe,
    get_support_distribution_dataframe,
)
from smartleads.generalising import generalise_data

## Specify parameters

### Project details

In [7]:
PROJECT_ID = 'smartleads-393408' # set on gcp
SERVICE_ACCOUNT_KEY_PATH = "./key.json" # from gcp

# big query client
client = bq.get_new_client(service_account_key_path=SERVICE_ACCOUNT_KEY_PATH)

### Hyperparameters

In [8]:
SCORING_CHUNK_MOD = 100
MIN_SUPPORT = 0.001
MIN_FEATURES_IN_RULE = 3
MAX_FEATURES_IN_RULE = 3
QUERIES_PER_CHUNK = 100
MIN_CONFIDENCE = 0.1
MIN_OBS = 5
MIN_LIFT = 1.1
MAX_RULES_PER_PRODUCT = 5000
QUANTILES_FOR_PLOTS = 10
MAX_OPTIONS_PER_CUSTOMER = 3

### `TARGET` & `FEATURES`

List of features to be kept

In [9]:
TARGET = "BENEFIT_OPTION"
TARGET_PREFIX = "T:"
FEATURES = ["*"]

In [6]:
# Generate new
RUN_UNIQUE_IDENTIFIER = uuid.uuid4().hex
RUN_NAME = "prod"
RUN_UNIQUE_IDENTIFIER

'9c3aeb8b31b94da29225b68bea908696'


--- 
# Setting table names

### Set run details with auto unique identifier

In [10]:
# Generate new
RUN_UNIQUE_IDENTIFIER = uuid.uuid4().hex
RUN_NAME = "prod"
RUN_UNIQUE_IDENTIFIER

# Use exisiting
#RUN_NAME = "dev"
#RUN_UNIQUE_IDENTIFIER = "c8fa5a4d4c2f41c9869e6c76fa4ccda1"

'02a960c65cd4468895d3c982a6461644'

## Datasets

### Define Datasets 

In [11]:
RAW_DATASET = f"raw_{RUN_NAME}_{RUN_UNIQUE_IDENTIFIER}"
INPUT_DATASET = f"input_{RUN_NAME}_{RUN_UNIQUE_IDENTIFIER}"
INTERIM_DATASET = f"interim_{RUN_NAME}_{RUN_UNIQUE_IDENTIFIER}"
TEMPORARY_DATASET = f"temp_{RUN_NAME}_{RUN_UNIQUE_IDENTIFIER}"
OUTPUT_DATSET = f"output_{RUN_NAME}_{RUN_UNIQUE_IDENTIFIER}"

### Create Datasets

In [12]:
datasets = [
    RAW_DATASET,
    INPUT_DATASET,
    INTERIM_DATASET,
    TEMPORARY_DATASET,
    OUTPUT_DATSET,
]


create_datasets(
    client,
    datasets,
    PROJECT_ID,
    location="US"
)

Creating datasets:   0%|          | 0/5 [00:00<?, ?it/s]

Created dataset: smartleads-393408.raw_prod_02a960c65cd4468895d3c982a6461644
Created dataset: smartleads-393408.input_prod_02a960c65cd4468895d3c982a6461644
Created dataset: smartleads-393408.interim_prod_02a960c65cd4468895d3c982a6461644
Created dataset: smartleads-393408.temp_prod_02a960c65cd4468895d3c982a6461644
Created dataset: smartleads-393408.output_prod_02a960c65cd4468895d3c982a6461644


## Raw Table

```
# TODO this is missing; check repo
```


## Input Tables

In [13]:
source_table = f'`{PROJECT_ID}.{RAW_DATASET}.recommendations_input`'
input_table = f'`{PROJECT_ID}.{INPUT_DATASET}.train_input`'
input_score_table = f'`{PROJECT_ID}.{INPUT_DATASET}.score_input`'

print(f"Ensure the following tables exist in gcp {PROJECT_ID}: \n* {source_table} \n* {input_table} \n* {input_score_table}")


Ensure the following tables exist in gcp smartleads-393408: 
* `smartleads-393408.raw_prod_02a960c65cd4468895d3c982a6461644.recommendations_input` 
* `smartleads-393408.input_prod_02a960c65cd4468895d3c982a6461644.train_input` 
* `smartleads-393408.input_prod_02a960c65cd4468895d3c982a6461644.score_input`


## Interim Tables

In [15]:
sample_table = f'`{PROJECT_ID}.{INTERIM_DATASET}.train_sample`'
engineered_table = f'`{PROJECT_ID}.{INTERIM_DATASET}.train_engineered`'
engineered_score_table = f'`{PROJECT_ID}.{INTERIM_DATASET}.score_engineered`'
long_table =  f'`{PROJECT_ID}.{INTERIM_DATASET}.feature_long`'
support_table = f'`{PROJECT_ID}.{INTERIM_DATASET}.support`'
id_agg_table = f'`{PROJECT_ID}.{INTERIM_DATASET}.id_agg`' # Customer level
id_agg_score_table = f'`{PROJECT_ID}.{INTERIM_DATASET}.score_id_agg`'
combo_count_table = f'`{PROJECT_ID}.{INTERIM_DATASET}.combo_count`'

## Temporary Tables

In [16]:
combo_temp_format = '`' + PROJECT_ID + f'.{TEMPORARY_DATASET}'+'.combos_{i}`'
scoring_temp_table_format = '`' + PROJECT_ID + f'.{TEMPORARY_DATASET}'+'.score_combos_{i}`'

## Output Tables

In [17]:
lift_table = f'`{PROJECT_ID}.{OUTPUT_DATSET}.lift`'
lift_filtered_table = f'`{PROJECT_ID}.{OUTPUT_DATSET}.lift_filtered`'
fully_scored_table = f'`{PROJECT_ID}.{OUTPUT_DATSET}.fully_scored`'
top_products_table = f'`{PROJECT_ID}.{OUTPUT_DATSET}.top_products`'
top_products_wide_table = f'`{PROJECT_ID}.{OUTPUT_DATSET}.top_products_wide`'

## Export table names

In [19]:
TABLES_META_DATA = {
    "source_table": source_table, #TODO! no value
    "input_table": input_table,
    "input_score_table": input_score_table,
    "sample_table": sample_table,
    "engineered_table": engineered_table,
    "engineered_score_table": engineered_score_table,
    "long_table": long_table,
    "support_table": support_table,
    "id_agg_table": id_agg_table,
    "id_agg_score_table": id_agg_score_table,
    "combo_count_table": combo_count_table,
    "combo_temp_format": combo_temp_format,
    "scoring_temp_table_format": scoring_temp_table_format,
    "lift_table": lift_table,
    "lift_filtered_table": lift_filtered_table,
    "fully_scored_table": fully_scored_table,
    "top_products_table": top_products_table,
    "top_products_wide_table": top_products_wide_table
}

if not os.path.exists("../META/"):
    os.makedirs('../META/')

TABLES_META_DATA_JSON = json.dumps(TABLES_META_DATA)

with open('../META/TABLES_META_DATA.json', 'w') as outfile:
    outfile.write(TABLES_META_DATA_JSON)

# Import your raw data

Rules for the data  
1. You have to have a field that is the `ID` for the customer  
2. You have to have your products or targets as an [array](https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays) in the `TARGET` field.  
3. `FEATURES` need to either be categorical already, ordinal as integers or arrays  
4. You've already converted your categorical columns to lower case  
5. You don't have any funny spaces or newlines in your data  

In [20]:
help(generalise_data)
generalise_data(client, FEATURES, input_table, source_table, TARGET)

Help on function generalise_data in module smartleads.generalising:

generalise_data(client: 'bigquery.Client', features: 'list', input_table: 'str', source_table: 'str', target: 'str')
    Generate a generalised data table.
    
    Args:
        client (bigquery.Client): A bigquery Client instance.
        features (list): list of features to be included in the general table.
        input_table (str): the input tabe name. It must inlcude the project id and dataset.
        source_table (str): the output tabe name. It must inlcude the project id and dataset.
        target (str): the target variable.



BadRequest: 400 Not found: Table smartleads-393408:raw_prod_02a960c65cd4468895d3c982a6461644.recommendations_input was not found in location US at [3:1]

Location: US
Job ID: fe7ffc2f-1151-4272-999f-fd8f1075ab7f


# Look at / profile your raw data

### Number of records in input table

In [ ]:
number_of_records_in_input_table = smartleads.profiling.get_num_rows(client, input_table)
number_of_records_in_input_table

### Max number of products per customer

In [ ]:
max_number_of_products_per_customer = smartleads.profiling.get_max_products_per_customer(client, input_table)
max_number_of_products_per_customer

### Frequency count of products

In [ ]:
product_count_df =  smartleads.profiling.get_frequency_product_count(client, input_table)
product_count_df

In [ ]:
len(product_count_df)

In [ ]:
import plotly.express as px
fig = px.bar(product_count_df, x='product', y='frequency')
fig.show()

# Sample your data

In [ ]:
# Sample your data manually with code
sql = (
    f'drop table if exists {sample_table}; '
    f'create table {sample_table} as '
    f'select * '
    f'from {input_table} ' 
    f'where OBSERVATION_DATE = DATE \'2020-06-30\' '
)
bq.execute_and_wait(client, sql)

# Specify your fields

## Specify your ID column

In [ ]:
ID_COLUMN = 'COVER_NUMBER'

## Columns to keep as is
These are columns where you believe you already have a good idea that they're basically right.  
They are already categorical, you don't have too many of them and you've already grouped together similar items.  
In here, you have the following inputs for each field:
* column_name: This is the name of the column in your input data
* prefix: This is the text that you want to go in front of the value from the column. So, if your age band is "18-25", then use "age" as a prefix makes the variable "age:18-25". This is required to make the baskets understandable later. Try to keep this as short as possible while retaining as much meaning as possible. Also, keep it lower case. 
* new_col_name: This is the new column name that BigQuery will use internally. Again, try keep it short and lower case.  

In [ ]:
COLUMNS_TO_KEEP_AS_IS = [
  {'column_name': 'AGEBAND','prefix': 'age', 'new_col_name': 'age', "drop_nulls": False, "map_nulls_to": "AGE_UNKNOWN" },
  {'column_name': 'GENDER', 'prefix': 'gender', 'new_col_name': 'gender', "drop_nulls": False, "map_nulls_to": "other"},
  {'column_name': 'RACE', 'prefix': 'race', 'new_col_name': 'race', "drop_nulls": False, "map_nulls_to": "AGE_UNKNOWN" },
  {'column_name': 'MARITAL_STATUS', 'prefix': 'marital', 'new_col_name': 'marital', "drop_nulls": False, "map_nulls_to": "unknown"},
  {'column_name': 'BENEFIT_UTILISATION_MAX_BAND', 'prefix': 'util', 'new_col_name': 'util', "drop_nulls": False, "map_nulls_to": "unknown"},
  {'column_name': 'EMPLOYER_NAME', 'prefix': 'employer', 'new_col_name': 'employer', "drop_nulls": False, "map_nulls_to": "unknown"},
  {'column_name': 'EMPLOYER_GROUP_TYPE', 'prefix': 'grp', 'new_col_name': 'grp', "drop_nulls": False, "map_nulls_to": "unknown"},
  {'column_name': 'EMPLOYER_INDUSTRY', 'prefix': 'industry', 'new_col_name': 'industry', "drop_nulls": False, "map_nulls_to": "unknown"}
]

In [ ]:
REMAP_CATEGORICAL_COLUMNS = [
    {
       'column_name': 'EMPLOYER_INDUSTRY', 
       'prefix':'sector',
       'new_col_name': 'sector',
       "drop_nulls": False,
       'categorical_maping': [
           {
               "new_catergorical_variable":"Government",
               "mapping_values": ['Health','Mining ','Food / Restaurants ','Automotive','Insurance ']
           }
       ]
    }
]

## Ordinal upper limit

These values generally start counting at zero, but need to be capped at a maximum value. 
So, for example, if someone belongs to a medical scheme, they might have 8 dependents. We would want to group that together as 4+.  
* column_name: Name of the column in the input data  
* upper_limit: Cut off point for the values  
* prefix: This is the text that you want to go in front of the value from the column. So, if your age band is "18-25", then use "age" as a prefix makes the variable "age:18-25". This is required to make the baskets understandable later. Try to keep this as short as possible while retaining as much meaning as possible. Also, keep it lower case. 
* new_col_name: This is the new column name that BigQuery will use internally. Again, try keep it short and lower case.  

In [ ]:
ORDINAL_UPPER_LIMIT = [
  {'column_name': 'DEPENDENT_COUNT_ADULT', 'upper_limit': 4, 'prefix': 'adult_dep', 'new_col_name': 'adult_dep', "drop_nulls": False},
  {'column_name': 'DEPENDENT_COUNT_CHILD', 'upper_limit': 4, 'prefix': 'child_dep', 'new_col_name': 'child_dep',"drop_nulls": False, "map_nulls_to": "unknown"},
  {'column_name': 'CHRONIC_CONDITION_COUNT', 'upper_limit': 4, 'prefix': 'chronic_count', 'new_col_name': 'chronic_count',"drop_nulls": False, "map_nulls_to": "unknown"},
]

## Manual columns

The manual columns refer to columns where you provide BigQuery code to manually add the columns.
* sql: The sql code to execute.  
* new_col_name: The new column name to use.  

In [ ]:
MANUAL_COLUMNS = [
    {
        'sql': 
        r'case '
        r'   when CLAIMS_NOT_PAID_ROLLING_12MONTH_PERCENTAGE is Null then "unpaid_claims: zz: Null" '
        r'   when CLAIMS_NOT_PAID_ROLLING_12MONTH_PERCENTAGE < 0 then "unpaid_claims: 00: Negative" '
        r'   when CLAIMS_NOT_PAID_ROLLING_12MONTH_PERCENTAGE < 0.05 then "unpaid_claims: 01: 0% - 5%" '
        r'   when CLAIMS_NOT_PAID_ROLLING_12MONTH_PERCENTAGE < 0.1 then "unpaid_claims: 02: 5% - 10%" '
        r'   else "unpaid_claims: 03: 10% + " '
        r'end as unpaid_claims ',
        'new_col_name': 'unpaid_claims'
    }
]

# Array columns

The array columns here actually refer to a comma separated string. If something is already specified as an array, you would keep the column as is. However, if it is a comma separated string, these values will be decomposed into their components.
* column_name: This is the name of the column in your input data
* prefix: This is the text that you want to go in front of the value from the column. So, if your age band is "18-25", then use "age" as a prefix makes the variable "age:18-25". This is required to make the baskets understandable later. Try to keep this as short as possible while retaining as much meaning as possible. Also, keep it lower case. 
* new_col_name: This is the new column name that BigQuery will use internally. Again, try keep it short and lower case.  

In [ ]:
ARRAY_COLUMNS = [
    {'column_name': 'CHRONIC_CONDITION_LIST_UNIQUE', 'prefix': 'ccl', 'new_col_name': 'ccl', "drop_nulls": False, "map_nulls_to": "unknown"}
]

# Engineer your data

In [ ]:
DEFAULT_MAP_NULLS_TO = "zz:Null"
preprocessor =  smartleads.preprocessing.Preprocessor(
    ID_COLUMN,
    REMAP_CATEGORICAL_COLUMNS,
    COLUMNS_TO_KEEP_AS_IS,
    ORDINAL_UPPER_LIMIT,
    MANUAL_COLUMNS,
    ARRAY_COLUMNS,
    sample_table,
    engineered_table,
    DEFAULT_MAP_NULLS_TO,
)
sql = preprocessor.get_engineered_sql()
smartleads.util.bq.execute_and_wait(client, sql)

# Calculate support

### Calculate Feature Long Table

In [ ]:
sql = smartleads.support.get_long_table_sql(
    long_table,
    engineered_table,
    COLUMNS_TO_KEEP_AS_IS,
    REMAP_CATEGORICAL_COLUMNS,
    ORDINAL_UPPER_LIMIT,
    MANUAL_COLUMNS,
    ARRAY_COLUMNS,
    TARGET_PREFIX
)

smartleads.util.bq.execute_and_wait(client, sql)

In [ ]:
sql = smartleads.support.get_support_sql(
    long_table, 
    support_table, 
    sample_table,
    MIN_SUPPORT,
    TARGET_PREFIX
)


smartleads.util.bq.execute_and_wait(client, sql)

In [ ]:
support_distribution = smartleads.support.get_support_distribution(
    client,
    support_table,
    QUANTILES_FOR_PLOTS
)

In [ ]:
support_distribution

Based on this, you can see that you get rid of half of your data by requiring tiny support.
I would say we could confidently say MIN_SUPPORT = 0.001

In [ ]:
support_distribution_products = smartleads.support.get_support_distribution_products(
    client,
    support_table,
    TARGET_PREFIX
)

In [ ]:
support_distribution_products

Note that we forced all products to be included above even though the support is so small

# Aggregating features to customer level

In [ ]:
sql = smartleads.id_agg.get_id_agg_sql(
    engineered_table, support_table, id_agg_table, 
    COLUMNS_TO_KEEP_AS_IS, REMAP_CATEGORICAL_COLUMNS, ORDINAL_UPPER_LIMIT, MANUAL_COLUMNS, ARRAY_COLUMNS,
    SCORING_CHUNK_MOD,
    TARGET_PREFIX
)
smartleads.util.bq.execute_and_wait(client, sql)

# Counting the number of times combinations occur

In [ ]:
sql = smartleads.combo_count.get_count_by_combination(1000, id_agg_table, combo_count_table)
smartleads.util.bq.execute_and_wait(client, sql)

# Generate all combinations

In [ ]:
min_features_per_id, max_features_per_id = \
    smartleads.combo_generation.get_feature_range(client, combo_count_table)

In [ ]:
smartleads.util.bq.clear_old_runs(client, f'{PROJECT_ID}.{TEMPORARY_DATASET}', [''])

In [ ]:
smartleads.combo_generation.generate_all_combinations(
    client, min_features_per_id, max_features_per_id,
    MIN_FEATURES_IN_RULE, MAX_FEATURES_IN_RULE,
    combo_count_table, QUERIES_PER_CHUNK,
    combo_temp_format
)

# Calculate lift

In [ ]:
sql = smartleads.lift.get_lift(
    lift_table, combo_temp_format, support_table, 
    MIN_CONFIDENCE, MIN_OBS, MIN_LIFT,
    TARGET_PREFIX)
smartleads.util.bq.execute_and_wait(client, sql)

In [ ]:
associations_df = smartleads.lift.get_associations_df(
    client,
    lift_table
)

In [ ]:
associations_df.style.format(thousands = ' ')

In [ ]:
associations_df.sum()

In [ ]:
sql = smartleads.lift.get_lift_filtered(
    lift_table, lift_filtered_table, MAX_RULES_PER_PRODUCT
)
smartleads.util.bq.execute_and_wait(client, sql)

## Insights

In [ ]:
sample_associations = get_sample_associations_df(client,lift_filtered_table)
sample_associations

In [ ]:
sample_associations.loc[0,:].to_dict()

In [ ]:
target_df = get_target_dataframe(client,lift_filtered_table)
target_df

In [ ]:
included_df = get_included_dataframe(client, lift_table)
included_df

In [ ]:
lift_distribution = get_lift_distribution_dataframe(
    client,
    lift_filtered_table,
    QUANTILES_FOR_PLOTS
)
lift_distribution

In [ ]:
confidence_distribution = get_confidence_distribution_dataframe(
    client,
    lift_table,
    QUANTILES_FOR_PLOTS
)
confidence_distribution

In [ ]:
support_distribution = get_support_distribution_dataframe(
    client,
    lift_table,
    QUANTILES_FOR_PLOTS
)
support_distribution

# Scoring

In [ ]:
sql = (
    f'DROP TABLE IF EXISTS {input_score_table}; '
    f'CREATE table {input_score_table} as '
    f'SELECT * '
    f'FROM {input_table} '
    f'where OBSERVATION_DATE = DATE \'2020-06-30\' '
)

bq.execute_and_wait(client, sql)

# Profile scoring data

In [ ]:
number_of_records_in_input_score_table = smartleads.profiling.get_num_rows(client, input_score_table)
number_of_records_in_input_score_table

# Engineer your data

In [ ]:
DEFAULT_MAP_NULLS_TO = "zz:Null"
preprocessor =  smartleads.preprocessing.Preprocessor(
    ID_COLUMN,
    REMAP_CATEGORICAL_COLUMNS,
    COLUMNS_TO_KEEP_AS_IS,
    ORDINAL_UPPER_LIMIT,
    MANUAL_COLUMNS,
    ARRAY_COLUMNS,
    input_score_table,
    engineered_score_table,
    DEFAULT_MAP_NULLS_TO,
)
sql = preprocessor.get_engineered_sql()
smartleads.util.bq.execute_and_wait(client, sql)

In [ ]:
sql = smartleads.id_agg.get_id_agg_sql(
    engineered_score_table, support_table, id_agg_score_table, 
    COLUMNS_TO_KEEP_AS_IS, REMAP_CATEGORICAL_COLUMNS, ORDINAL_UPPER_LIMIT, MANUAL_COLUMNS, ARRAY_COLUMNS,
    SCORING_CHUNK_MOD,
    TARGET_PREFIX
)
smartleads.util.bq.execute_and_wait(client, sql)

In [ ]:
smartleads.util.bq.clear_old_runs(client, f'{PROJECT_ID}.{TEMPORARY_DATASET}', [''])

In [ ]:
jobs = smartleads.score.get_fully_scored(
    client,
    fully_scored_table,
    id_agg_score_table,
    lift_filtered_table,
    scoring_temp_table_format,
    SCORING_CHUNK_MOD
)

In [ ]:

number_of_scored_customers = smartleads.score.get_number_of_scored_customers(
    client, 
    fully_scored_table
)

In [ ]:
number_of_scored_customers

In [ ]:

percentage_of_scored_customers_per_rule = smartleads.score.get_percentage_of_scored_customers_per_rule(
    client,
    number_of_scored_customers,
    fully_scored_table
)

In [ ]:
percentage_of_scored_customers_per_rule

In [ ]:
sql = smartleads.score.get_top_products(top_products_table, fully_scored_table, MAX_OPTIONS_PER_CUSTOMER)

In [ ]:
smartleads.util.bq.execute_and_wait(client, sql)

In [20]:
right_option_dataframe = smartleads.score.get_right_option_dataframe(
    client,
    top_products_table,
    TARGET_PREFIX
)

In [21]:
right_option_dataframe.style\
    .format('{:,.2%}'.format)\
    .background_gradient(cmap = 'Blues', low=0, high=1)

rule_option,Copper Core,Essential,Gold Ascend,Gold Ascend EDO,Platinum Enhanced,Platinum Enhanced EDO,Plus,Silver Saver,Titanium Executive,Value
customer_option,,,,,,,,,,
Access,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Affordable Care,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Copper Core,22.31%,24.45%,0.00%,4.97%,0.00%,1.34%,4.32%,26.08%,3.26%,13.28%
Essential,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Gold Ascend,1.73%,20.86%,0.10%,5.25%,0.32%,13.30%,5.37%,22.27%,11.89%,18.90%
Gold Ascend EDO,2.29%,9.15%,0.00%,50.88%,0.00%,9.15%,1.41%,11.80%,8.63%,6.69%
HOSMED - STEP,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Platinum Enhanced,0.68%,17.85%,0.00%,3.10%,2.34%,18.68%,5.92%,12.72%,24.80%,13.91%
Platinum Enhanced EDO,1.36%,4.42%,0.00%,9.18%,0.00%,62.24%,1.36%,5.10%,11.56%,4.76%


In [ ]:
right_option_summary = smartleads.score.get_right_option_summary(
    client,
    top_products_table
)

In [ ]:
right_option_summary

In [ ]:
right_option_summary/number_of_scored_customers

In [ ]:
sql = smartleads.score.get_top_products_wide(top_products_table, top_products_wide_table, MAX_OPTIONS_PER_CUSTOMER)
smartleads.util.bq.execute_and_wait(client, sql)

In [ ]:
product = 'T:Platinum Enhanced Plan'
num_customers = 100
customers_for_product_df = smartleads.score.get_product_recommendation_dataframe(
    client,
    product,
    num_customers,
    top_products_table,
)

In [ ]:
for rule in customers_for_product_df['rule_feature_array'].head(3).tolist():
    print('-------')
    for r in rule:
        print(r)